In [12]:
import os
import nest_asyncio

from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.vector_stores import VectorStoreQueryResult
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import Settings
from typing import List
from dotenv import load_dotenv
import json
import os
nest_asyncio.apply()
load_dotenv()

True

In [13]:
from docx import Document

def extract_tables_as_markdown(docx_path):
    doc = Document(docx_path)
    markdown_tables = []
    for table in doc.tables:
        rows = []
        for row in table.rows:
            cells = [cell.text.strip() for cell in row.cells]
            rows.append("| " + " | ".join(cells) + " |")
        if rows:
            header = rows[0]
            separator = "| " + " | ".join(["---"] * len(table.columns)) + " |"
            markdown_table = "\n".join([header, separator] + rows[1:])
            markdown_tables.append(markdown_table)
    return markdown_tables

In [14]:
from llama_index.readers.file import DocxReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import Document
import os
import glob

# ——————————————
# CONFIGURATION
# ——————————————

# 1) Directory where all your .docx files live
DOCX_FOLDER = "documents/"

# ——————————————
# STEP 1: Discover all .docx files
# ——————————————

all_paths = glob.glob(os.path.join(DOCX_FOLDER, "*.docx"))
print(f"Found {len(all_paths)} .docx file(s):")
for p in all_paths:
    print("  •", p)

# ——————————————
# STEP 2: Load each DOCX and wrap as Document
# ——————————————

reader = DocxReader()
raw_documents = []
for file_path in all_paths:
    # load_data returns a list of in‐memory “page” objects 
    docx_pages = reader.load_data(file_path)
    for page_obj in docx_pages:
        raw_documents.append(
            Document(
                text=page_obj.text,
                metadata={"source": os.path.basename(file_path)}
            )
        )

print(f"Loaded {len(raw_documents)} raw Document(s) from all .docx files.")

# ——————————————
# STEP 3: Chunk each Document semantically
# ——————————————

splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=100)

nodes = []
for doc in raw_documents:
    chunks = splitter.split_text(doc.text)
    for i, chunk in enumerate(chunks):
        nodes.append(
            Document(
                text=chunk,
                metadata={**doc.metadata, "chunk_id": i}
            )
        )

print(f"After splitting, we have {len(nodes)} chunked Documents (nodes).")

# ——————————————
# FINAL: Assign to `documents` so the rest of your pipeline can stay unchanged
# ——————————————

documents = nodes

# Now you can call your index creation exactly as before:
# from llama_index.core.embeddings import CohereEmbedding  # or whichever embed_model you use
# from llama_index.core.storage import StorageContext
# from llama_index.vector_stores import QdrantVectorStore
# from llama_index import VectorStoreIndex

# Example (adjust embed_model, storage_context, etc. to your configuration):
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents=documents,
#     embed_model=Settings.embed_model,
#     storage_context=storage_context,
# )

Found 7 .docx file(s):
  • documents/FA-G-02_StaffReimbursement_SectionAware.docx
  • documents/CPAX-FN-005_ProjectInvestment_SectionAware.docx
  • documents/FA-G-15_SectionAware_Final_Sectioned.docx
  • documents/FA-G-17_SectionAwareChunking.docx
  • documents/FA-G-07_NonTradeSupplier_SectionAware.docx
  • documents/FA-B2B-01_CreditMgmt_SectionAware.docx
  • documents/FA-G-08 อำนาจอนุมัติรายจ่ายสำหรับ Purchase Requisition_แปลงตาราง.docx
Loaded 7 raw Document(s) from all .docx files.
After splitting, we have 122 chunked Documents (nodes).


In [15]:
from llama_index.core import StorageContext

## Setup Cohear Embedding service

In [16]:
# … (no need to call load_dotenv() here) …

# Hard-code your key and model ID:
COHEAR_KEY      = "Iyn2rmOdEgiUKfxptJDhCKRwgfeIWhZ37sxzKUAc"
COHEAR_MODEL_ID = "embed-multilingual-light-v3.0"

print("🔑 Using Cohere key:   ", COHEAR_KEY)
print("🔢 Using Cohere model: ", COHEAR_MODEL_ID)

embed_model = CohereEmbedding(
    api_key=COHEAR_KEY,
    model_name=COHEAR_MODEL_ID,
    input_type="search_document",
    embedding_type="float",
)

Settings.chunk_size = 1024

🔑 Using Cohere key:    Iyn2rmOdEgiUKfxptJDhCKRwgfeIWhZ37sxzKUAc
🔢 Using Cohere model:  embed-multilingual-light-v3.0


## Innitiates VectorStore database (Qdrant)

In [17]:
# creates a persistant index to disk
client = QdrantClient(url="http://localhost:6334", api_key=os.getenv("QDRANT_API_KEY"),  prefer_grpc=True)

# # delete collection if it exists
if client.collection_exists(os.getenv("QDANT_COLLENCTION_NAME")):
    client.delete_collection(os.getenv("QDANT_COLLENCTION_NAME"))

# create our vector store with hybrid indexing enabled
vector_store = QdrantVectorStore(
    os.getenv("QDANT_COLLENCTION_NAME"),
    client=client,
    enable_hybrid=True,
    batch_size=20,
    prefer_grpc=True,
)


/var/folders/z5/qxv1ygl57v34c8nqkcj1m3200000gn/T/ipykernel_10595/2846219684.py:2: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(url="http://localhost:6334", api_key=os.getenv("QDRANT_API_KEY"),  prefer_grpc=True)


## Start embedding process.... into vector database

In [18]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=documents, embed_model=embed_model, storage_context=storage_context,
)
COHERE_KEY = os.getenv("COHEAR_API_KEY")
COHEAR_MODEL = os.getenv("COHEAR_MODEL_ID")
QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6334")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY", None)
COLLECTION_NAME = os.getenv("QDANT_COLLENCTION_NAME", "my_collection")

if not COHERE_KEY:
    raise RuntimeError("COHERE_API_KEY is not set in your environment")

# 2) (Re)initialize the Cohere embedder with the new API key
embed_model = CohereEmbedding(
    cohere_api_key=COHERE_KEY,
    model_name=COHEAR_MODEL,
    input_type="search_document",
    embedding_type="float",
)

## Try to retrive relavent nodes with question.

In [19]:
embed_model = CohereEmbedding(
    api_key=os.getenv("COHEAR_API_KEY"),
    model_name=os.getenv("COHEAR_MODEL_ID"),
    input_type="search_query",
    embedding_type="float",
)

search_query_retriever = index.as_retriever()

search_query_retrieved_nodes = search_query_retriever.retrieve(
"Do all Walmart locations offer scan & go?"
)

In [20]:
from llama_index.core.response.notebook_utils import display_source_node
for n in search_query_retrieved_nodes:
    display_source_node(n, source_length=2000)

**Node ID:** 0d70a22c-d297-4d7b-a5e1-73705e22ba8b<br>**Similarity:** 0.17325928807258606<br>**Text:** สายบังคับบัญชาของทีมขาย B2B

พนักงานขายของ Lotus’s : 

Go-fresh : ผู้จัดการทั่วไป (Area General Manager – AGM)

Hypermarket พนักงานขายในสาขา : ผู้จัดการสาขา (Store Manager) ->ผู้จัดการทั่วไป (Area General Manager – AGM)

Hypermarket พนักงานขายนอกสาขา : ผู้จัดการเขตขาย (Zone Manager) -> ผู้จัดการอาวุโสเขตขาย (Senior Zone Manager)

พนักงานขายของ Makro : ผู้จัดการฝ่ายขาย (Sales Manager) ->ผู้จัดการฝ่ายขายประจำภูมิภาค (Regional Sales Manager)






คำถามอื่นๆของ B2B ที่นอกเหนือ Policy

หัวข้อคำถามเกี่ยวกับเรื่อง  Open new CV - เปิดหน้าบัญชีลูกค้าใหม่  คำถาม : ลูกค้ารายนี้เคยเปิดหน้าบัญชี หรือเคยมีการซื้อขายกับบริษัทฯมาก่อนหรือไม่?  คำตอบ : ขอให้ตรวจสอบข้อมูลของลูกค้าโดยใช้เลขประจำตัวผู้เสียภาษี 13 หลัก (Tax ID) เข้าไปตรวจสอบในระบบ smartsoft

หัวข้อคำถามเกี่ยวกับเรื่อง  Open new CV - เปิดหน้าบัญชีลูกค้าใหม่  คำถาม : ต้องใช้เอกสารอะไรบ้างในการเปิดบัญชีลูกค้าใหม่?  คำตอบ : กรณีลูกค้าเป็นบุคคลธรรมดาต้องแนบเอกสารสำคัญดังนี้ สำเนาใบเปิดบัญชีลูกค้า+สำเนาบัตรประจำตัวประชาชนหรือบัตรข้าราชการของลูกค้า/เจ้าของ/ผู้ประกอบการ/หุ้นส่วนผู้จัดการ/กรรมการผู้มีอำนาจ+รูปถ่ายเซลฟี่ของพนักงานขายกับสถานประกอบการ (ต้องเห็นป้ายหน้าร้าน/บริษัทฯ) และหากมีเอกสารเหล่านี้ให้แนบมาด้วย คือสำเนาหนังสือรับรองการจดทะเบียนพาณิชย์หรือสำเนาหังสือจัดตั้งหุ้นส่วนสามัญและ/หรือสำเนาใบทะเบียนภาษีมูลค่าเพิ่ม(ภพ.<br>

**Node ID:** e1a6da9c-c96f-4e04-bf77-8dd6de24cf92<br>**Similarity:** 0.1716158092021942<br>**Text:** ถ่ายรูปตัวเองอยู่หน้าร้านไม่ต้องใส่แมส 2.อย่าแนบรูปกลับด้าน (Mirror) 3.สวมเสื้อพนักงานหรือแขวนบัตรพนักงาน

หัวข้อคำถามเกี่ยวกับเรื่อง  Open new CV - เปิดหน้าบัญชีลูกค้าใหม่  คำถาม : กรณีที่พนักงานขายไปพบลูกค้าเพียงคนเดียว จะถ่ายรูปกับสถานประกอบการอย่างไร  คำตอบ : สามารถแนบรูปเซลฟี่ตนเองหน้าสถานประกอบการ และ แนบพร้อมกับรูปถ่ายของตึกอาคารมุมกว้างเพิ่ม
ถ่ายรูปให้เห็นโลโก้ป้าย/หน้าร้าน/บ้านเลขที่ 1.ถ่ายรูปตัวเองอยู่หน้าร้านไม่ต้องใส่แมส 2.อย่าแนบรูปกลับด้าน (Mirror) 3.สวมเสื้อพนักงานหรือแขวนบัตรพนักงาน

หัวข้อคำถามเกี่ยวกับเรื่อง  Open new CV - เปิดหน้าบัญชีลูกค้าใหม่  คำถาม : กลุ่มลูกค้าใดบ้างที่ต้องถ่ายกับ พนักงานกับ สถานประกอบการ  คำตอบ : กลุ่มลูกค้าเครดิตระยะสั้น กลุ่มเครดิตบุคคลธรรมดา 0-3 วัน และ นิติบุคคล 0-30 วัน หรือ ประเภทลูกค้า Type 70-80 

หัวข้อคำถามเกี่ยวกับเรื่อง  Open new CV - เปิดหน้าบัญชีลูกค้าใหม่  คำถาม : ลูกค้า COD สด Instore คืออะไร   คำตอบ : สำหรับการขายสดในร้าน  Cash on Delivery (COD – Instore)  ข้อมูลลูกค้าจะถูกดึงมาจาก My Lotus’s การสร้างลูกค้าใหม่บุคคลธรรมดาจะไม่ผ่านการอนุมัติ CV โดยผู้อนุมัติฝ่ายขายและ MDM แต่พนักงานขายยังคงต้องแนบสำเนาบัตรประชาชนของลูกค้าเป็นเอกสารประกอบในระบบ (ในกรณีที่ลูกค้ายังไม่ได้เป็นสมาชิก ลูกค้าต้องดำเนินการสมัคร user My Lotus’s<br>